In [43]:
import datetime
import numpy as np
import matplotlib.colors as colors
import matplotlib.finance as finance
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import csv

#csv file needs column headers: date, open, high, low, close, volume, adj_close
#in reverse date order so most recent date on top
r = mlab.csv2rec('C:/Users/msteinme/Documents/RSImay.csv')
r.sort()
today = datetime.date.today()
def moving_average(x, n, type='simple'):
    """
    compute an n period moving average.
    type is 'simple' | 'exponential'
    """
    x = np.asarray(x)
    if type == 'simple':
        weights = np.ones(n)
    else:
        weights = np.exp(np.linspace(-1., 0., n))

    weights /= weights.sum()

    a = np.convolve(x, weights, mode='full')[:len(x)]
    a[:n] = a[n]
    return a


def relative_strength(prices, n=14):
    """
    compute the n period relative strength indicator
    http://stockcharts.com/school/doku.php?id=chart_school:glossary_r#relativestrengthindex
    http://www.investopedia.com/terms/r/rsi.asp
    """

    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1. + rs)

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # cause the diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n - 1) + upval)/n
        down = (down*(n - 1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1. + rs)

    return rsi


def moving_average_convergence(x, nslow=26, nfast=12):
    """
    compute the MACD (Moving Average Convergence/Divergence) using a fast and slow exponential moving avg'
    return value is emaslow, emafast, macd which are len(x) arrays
    """
    emaslow = moving_average(x, nslow, type='exponential')
    emafast = moving_average(x, nfast, type='exponential')
    return emaslow, emafast, emafast - emaslow


plt.rc('axes', grid=True)
plt.rc('grid', color='0.75', linestyle='-', linewidth=0.5)

textsize = 9
left, width = 0.1, 0.8
rect1 = [left, 0.7, width, 0.2]
rect2 = [left, 0.3, width, 0.4]
rect3 = [left, 0.1, width, 0.2]


fig = plt.figure(facecolor='white')
axescolor = '#f6f6f6'  # the axes background color

ax1 = fig.add_axes(rect1, axisbg=axescolor)  # left, bottom, width, height
ax2 = fig.add_axes(rect2, axisbg=axescolor, sharex=ax1)
ax2t = ax2.twinx()
ax3 = fig.add_axes(rect3, axisbg=axescolor, sharex=ax1)


# plot the relative strength index
prices = r.adj_close
rsi = relative_strength(prices)
fillcolor = 'darkgoldenrod'

ax1.plot(r.date, rsi, color=fillcolor)
ax1.axhline(70, color='red')
ax1.axhline(30, color='green')
ax1.fill_between(r.date, rsi, 70, where=(rsi >= 70), facecolor=fillcolor, edgecolor=fillcolor)
ax1.fill_between(r.date, rsi, 30, where=(rsi <= 30), facecolor=fillcolor, edgecolor=fillcolor)
ax1.text(0.6, 0.9, '>70 = overbought (sell)', va='top', transform=ax1.transAxes, fontsize=textsize)
ax1.text(0.6, 0.1, '<30 = oversold (buy)', transform=ax1.transAxes, fontsize=textsize)
ax1.set_ylim(0, 100)
ax1.set_yticks([30, 70])
ax1.text(0.025, 0.95, 'RSI (14)', va='top', transform=ax1.transAxes, fontsize=textsize)
#ax1.set_title('ticker interested in')

# plot the price and volume data
dx = r.adj_close - r.close
low = r.low + dx
high = r.high + dx

deltas = np.zeros_like(prices)
deltas[1:] = np.diff(prices)
up = deltas > 0
ax2.vlines(r.date[up], low[up], high[up], color='black', label='_nolegend_')
ax2.vlines(r.date[~up], low[~up], high[~up], color='black', label='_nolegend_')
ma20 = moving_average(prices, 20, type='simple')
ma50 = moving_average(prices, 50, type='simple')

linema20, = ax2.plot(r.date, ma20, color='blue', lw=2, label='MA (20)')
linema50, = ax2.plot(r.date, ma50, color='red', lw=2, label='MA (50)')


last = r[-1]
s = '%s O:%1.2f H:%1.2f L:%1.2f C:%1.2f, V:%1.1fM Chg:%+1.2f' % (
    today.strftime('%d-%b-%Y'),
    last.open, last.high,
    last.low, last.close,
    last.volume*1e-6,
    last.close - last.open)
t4 = ax2.text(0.3, 0.9, s, transform=ax2.transAxes, fontsize=textsize)

props = font_manager.FontProperties(size=10)
leg = ax2.legend(loc='center left', shadow=True, fancybox=True, prop=props)
leg.get_frame().set_alpha(0.5)


volume = (r.close*r.volume)/1e6  # dollar volume in millions
vmax = volume.max()
poly = ax2t.fill_between(r.date, volume, 0, label='Volume', facecolor=fillcolor, edgecolor=fillcolor)
ax2t.set_ylim(0, 5*vmax)
ax2t.set_yticks([])


# compute the MACD indicator
fillcolor = 'darkslategrey'
nslow = 26
nfast = 12
nema = 9
emaslow, emafast, macd = moving_average_convergence(prices, nslow=nslow, nfast=nfast)
ema9 = moving_average(macd, nema, type='exponential')
ax3.plot(r.date, macd, color='black', lw=2)
ax3.plot(r.date, ema9, color='blue', lw=1)
ax3.set_ylim([-100,100])
ax3.fill_between(r.date, macd - ema9, 0, alpha=0.5, facecolor=fillcolor, edgecolor=fillcolor)


ax3.text(0.025, 0.95, 'MACD (%d, %d, %d)' % (nfast, nslow, nema), va='top',
         transform=ax3.transAxes, fontsize=textsize)

#ax3.set_yticks([])
# turn off upper axis tick labels, rotate the lower ones, etc
for ax in ax1, ax2, ax2t, ax3:
    if ax != ax3:
        for label in ax.get_xticklabels():
            label.set_visible(False)
    else:
        for label in ax.get_xticklabels():
            label.set_rotation(30)
            label.set_horizontalalignment('right')

    ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')


class MyLocator(mticker.MaxNLocator):
    def __init__(self, *args, **kwargs):
        mticker.MaxNLocator.__init__(self, *args, **kwargs)

    def __call__(self, *args, **kwargs):
        return mticker.MaxNLocator.__call__(self, *args, **kwargs)

# at most 5 ticks, pruning the upper and lower so they don't overlap
# with other ticks
#ax2.yaxis.set_major_locator(mticker.MaxNLocator(5, prune='both'))
#ax3.yaxis.set_major_locator(mticker.MaxNLocator(5, prune='both'))

ax2.yaxis.set_major_locator(MyLocator(5, prune='both'))
ax3.yaxis.set_major_locator(MyLocator(5, prune='both'))

plt.show()

In [23]:
rsi

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  3,  2,  2,  2,  2,  2,  2, 11, 23, 39, 47, 51,
       51, 51, 51, 51, 55, 55, 48, 48, 45, 45, 41, 37, 33, 27, 27, 24, 24,
       21, 19, 23, 23, 27, 27, 27, 27, 27, 43, 47, 47, 53, 53, 65, 65, 67,
       70, 70, 70, 76, 69, 73, 73, 76, 79, 79, 79, 83, 85, 89, 91, 92, 94,
       94, 94, 83, 83, 83, 77, 77, 72, 72, 72, 65, 65, 58, 44, 40, 54, 69,
       70, 75, 73, 73, 73, 62, 58, 59, 57, 58, 54, 54, 51, 49, 50, 50, 55,
       56, 56, 56, 56, 54, 54, 54, 52, 62, 70, 70, 81, 87, 88, 89, 86, 82,
       78, 72, 72, 72, 72, 65, 59, 55, 50, 45, 41, 31, 27, 22, 20, 20, 16,
       14, 16, 18, 24, 24, 24, 23, 23, 25, 25, 25, 25, 25, 28, 30, 36, 41,
       43, 41, 41, 38, 37, 34, 34, 34, 34, 36, 35, 35, 35, 31, 29, 29, 29,
       28, 24, 24, 22, 29, 29, 29, 29, 29, 29, 29, 29, 29, 51, 57, 67, 75,
       80, 81, 81, 78, 78, 69, 63, 63, 65, 71, 69, 71, 66, 66, 67, 67, 56,
       59, 59, 59, 59, 53

In [24]:
import pandas as pd
rsi = np.array(rsi)
rsi = pd.DataFrame(rsi)
rsi.columns = ['RSI']
rsi.head(5)

,RSI
0,0
1,0
2,0
3,0
4,0


In [29]:
data = pd.read_csv("C:/Users/msteinme/Documents/RSImay.csv")
data.head()

,date,open,high,low,close,volume,adj_close
0,6/13/2016,180,180,180,180,1361,180
1,6/6/2016,180,180,189,180,1361,180
2,5/30/2016,189,189,200,189,1360,189
3,5/23/2016,200,200,212,200,1359,200
4,5/16/2016,212,212,225,212,1358,212


In [19]:
rsi.to_csv("C:/Users/msteinme/Documents/rsiresults.csv")

In [32]:
data['RSI'] = rsi
data.head()

,date,open,high,low,close,volume,adj_close,RSI
0,6/13/2016,180,180,180,180,1361,180,0
1,6/6/2016,180,180,189,180,1361,180,0
2,5/30/2016,189,189,200,189,1360,189,0
3,5/23/2016,200,200,212,200,1359,200,0
4,5/16/2016,212,212,225,212,1358,212,0
